# Template

> Template related functions

In [ ]:
#| default_exp template

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from chem_templates.imports import *
from chem_templates.utils import *
from chem_templates.chem import Molecule
from chem_templates.filter import Filter, FilterResult

In [ ]:
#| export

class TemplateResult():
    def __init__(self, 
                 result:         bool, 
                 filter_results: list[bool], 
                 filter_data:    list[Union[FilterResult, None]]):
        
        self.result = result
        self.filter_results = filter_results
        self.filter_data = filter_data

class Template():
    def __init__(self, filters: list[Filter]):
        self.filters = filters
        
    def _empty_result(self):
        filter_results = [False for i in self.filters]
        filter_data = [None for i in self.filters]
        
        return TemplateResult(False, filter_results, filter_data)
        
    def __call__(self, molecule: Molecule, early_exit: bool=True) -> TemplateResult:
        results = self._empty_result()
        
        for i, f in enumerate(self.filters):
            res = f(molecule)
            results.filter_results[i] = res.filter_result
            results.filter_data[i] = res

            if (not res.filter_result) and early_exit:
                break
                
        results.result = all(results.filter_results)
                        
        return results

In [ ]:
from chem_templates.filter import *
from chem_templates.chem import mol_func_wrapper
from rdkit.Chem import rdMolDescriptors, Descriptors

hbd = mol_func_wrapper(rdMolDescriptors.CalcNumHBD)
hba = mol_func_wrapper(rdMolDescriptors.CalcNumHBA)
molwt = mol_func_wrapper(rdMolDescriptors.CalcExactMolWt)
logp = mol_func_wrapper(Descriptors.MolLogP)

filters = [
    ValidityFilter(),
    SingleCompoundFilter(),
    RangeFunctionFilter(hbd, 'hydrogen_bond_donors', None, 5),
    RangeFunctionFilter(hba, 'hydrogen_bond_acceptors', None, 10),
    RangeFunctionFilter(molwt, 'molecular_weight', None, 150),
    RangeFunctionFilter(logp, 'CLogP', None, 5)
]

template = Template(filters)

molecule = Molecule('c1ccccc1N=Nc1ccccc1')

res = template(molecule, early_exit=True)
assert res.result == False
assert res.filter_results == [True, True, True, True, False, False]

res = template(molecule, early_exit=False)
assert res.result == False
assert res.filter_results == [True, True, True, True, False, True]

assert template(Molecule('c1ccccc1N=N')).result

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()